In [1]:
# import packages
import glob
from pathlib import Path
import os
import geopandas as gpd
import iris
import iris.pandas
import numpy as np
from pathos.threading import ThreadPool as Pool
from datetime import datetime
from datetime import timedelta
# import cartopy
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
work_dir=Path("/mnt/u/LSM root zone/global_sr")

In [ ]:
#%% packages
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from netCDF4 import Dataset
import xarray as xr
import rioxarray
from shapely.geometry import mapping
from rasterstats import zonal_stats
import rasterio
from shapely.ops import snap
import grid_catchments_function as f
import netCDF4



#%% Hargreaves and Samani function
# EXTRA TERRESTRIAL RADIATION
# Duffie and Beckman 1991 (https://scottishsnow.wordpress.com/2013/05/20/ext-radiation/)
# Ra: daily total extraterrestrial radiation [MJ m-2 d-1]
# G: solar constant [MJ m-2 min-1]
# d: relative distance Earth-Sun [-]
# delta: solar declination [rad]
# J: number of day in the year
# phi: latitude (negative for SH) [rad]
# w: sunset hour angle [rad]

# HARGREAVES AND SAMANI EP
# Re: daily total extraterrestrial radiation [MJ m-2 d-1]
# labda: 2.26 [MJ kg-1]
# Temperature: [C]
# kg m-2 day-1 = mm day-1
    
def EP_HS(J,lat,Tmax,Tmin,Tav):
    # extraterrestrial radation
    G = 0.0820
    phi = lat/360 * (2*np.pi)
    d = 1 + 0.033 * np.cos(0.0172 * J)
    delta = 0.409 * np.sin(0.0172 * J - 1.39)
    w = np.arccos(-np.tan(phi)*np.tan(delta))
    Re = (24*60)/np.pi * G * d * (w*np.sin(phi)*np.sin(delta) + np.cos(phi)*np.cos(delta)*np.sin(w))
    
    # Hargreaves and Samani EP
    labda = 2.26 # MJ/kg
    alpha = 0.0023
    Ep = (1/labda) * alpha * Re * (Tav + 17.8) * np.sqrt(Tmax-Tmin) #convert to mm
    if(Ep<0):
        Ep=0
    return(Ep)

#%% get EP HS from T data GSWP
folder = 'U:/LSM root zone/global_sr_data/brazil/gswp_timeseries/1951_2010/'
br_attributes = pd.read_csv('C:/Users/fransjevanoors/surfdrive/Fransje/global_SR/data_br/br_attributes.csv',index_col=0)
catch_id = br_attributes.index

for i in range(len(catch_id)):
    a = pd.read_csv(str(folder)+'/'+str(catch_id[i])+'.csv',index_col=0)
    a.index = pd.to_datetime(a.index)
    
    day_in_year = []
    for j in a.index:
        day_in_year.append(j.timetuple().tm_yday) # convert datetime in day of year

    ep_array = []
    lat = br_attributes.gauge_lat.iloc[i]
    for l in range(len(a)):
        J = day_in_year[l]
        Ep = EP_HS(J,lat,Tmax=a.tasmax[l],Tmin=a.tasmin[l],Tav=a.tas[l]) 
        ep_array.append(Ep)
    a['ep_hs'] = ep_array
    a.to_csv(str(folder)+str(catch_id[i])+'.csv')
    print(catch_id[i])